This is a different approach to the cartpole question. I use Proximal Policy Optimation (PPO) to solve the cartpole problem. The actor maps the observation and the critic provides a value for the expected reward. The policy is updated with a stochastic gradient ascent optimizer. A stochastic gradient descent alogorithm fits the value function. Source: https://keras.io/examples/rl/ppo_cartpole/#visualizations   

In [1]:
!pip install numpy tensorflow keras gymnasium scipy matplotlib  #only works with python11 or older

In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
from keras import layers

import numpy as np
import tensorflow as tf
import gymnasium as gym
import scipy.signal

In [3]:
#agent values future rewards but at a discounted rate.
def discounted_cumulative_sums(x, discount):
    # Discounted cumulative sums of vectors for computing rewards-to-go and advantage estimates
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

In [4]:
#storing agents experiences so we can use them later for training.
class Buffer:
    # Buffer for storing trajectories
    def __init__(self, observation_dimensions, size, gamma=0.99, lam=0.95):
        # Buffer initialization
        self.observation_buffer = np.zeros(
            (size, observation_dimensions), dtype=np.float32
        )
        self.action_buffer = np.zeros(size, dtype=np.int32)
        self.advantage_buffer = np.zeros(size, dtype=np.float32)
        self.reward_buffer = np.zeros(size, dtype=np.float32)
        self.return_buffer = np.zeros(size, dtype=np.float32)
        self.value_buffer = np.zeros(size, dtype=np.float32)
        self.logprobability_buffer = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.pointer, self.trajectory_start_index = 0, 0

    def store(self, observation, action, reward, value, logprobability):
        # Append one step of agent-environment interaction
        if self.pointer >= len(self.observation_buffer):  # Prevent overflow
            print(f"Warning: Buffer is full. Resetting pointer.")
            return  # Stop storing if buffer is full
        self.observation_buffer[self.pointer] = observation
        self.action_buffer[self.pointer] = action
        self.reward_buffer[self.pointer] = reward
        self.value_buffer[self.pointer] = value
        self.logprobability_buffer[self.pointer] = logprobability
        self.pointer += 1

    def finish_trajectory(self, last_value=0):
        # Finish the trajectory by computing advantage estimates and rewards-to-go
        path_slice = slice(self.trajectory_start_index, self.pointer)
        rewards = np.append(self.reward_buffer[path_slice], last_value)
        values = np.append(self.value_buffer[path_slice], last_value)

        deltas = rewards[:-1] + self.gamma * values[1:] - values[:-1]

        self.advantage_buffer[path_slice] = discounted_cumulative_sums(
            deltas, self.gamma * self.lam
        )
        self.return_buffer[path_slice] = discounted_cumulative_sums(
            rewards, self.gamma
        )[:-1]

        self.trajectory_start_index = self.pointer

    def get(self):
        # Get all data of the buffer and normalize the advantages
        self.pointer, self.trajectory_start_index = 0, 0
        advantage_mean, advantage_std = (
            np.mean(self.advantage_buffer),
            np.std(self.advantage_buffer),
        )
        self.advantage_buffer = (self.advantage_buffer - advantage_mean) / advantage_std
        return (
            self.observation_buffer,
            self.action_buffer,
            self.advantage_buffer,
            self.return_buffer,
            self.logprobability_buffer,
        )

In [5]:
#creating feedforward nuerel network
def mlp(x, sizes, activation=keras.activations.tanh, output_activation=None):
    # Build a feedforward neural network
    for size in sizes[:-1]:
        x = layers.Dense(units=size, activation=activation)(x)
    return layers.Dense(units=sizes[-1], activation=output_activation)(x)

In [6]:
def logprobabilities(logits, a):
    # Compute the log-probabilities of taking actions a by using the logits (i.e. the output of the actor)
    logprobabilities_all = keras.ops.log_softmax(logits)
    logprobability = keras.ops.sum(
        keras.ops.one_hot(a, num_actions) * logprobabilities_all, axis=1
    )
    return logprobability


seed_generator = keras.random.SeedGenerator(1337)

In [7]:
#takes an observation and provides feedback from the policy.
@tf.function
def sample_action(observation):
    logits = actor(observation)
    action = keras.ops.squeeze(
        keras.random.categorical(logits, 1, seed=seed_generator), axis=1
    )
    return logits, action

In [8]:
#Actor: trains the policy -- ppo clips policy to prevent large updates that will destabilize learning.
@tf.function
def train_policy(
    observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        ratio = keras.ops.exp(
            logprobabilities(actor(observation_buffer), action_buffer)
            - logprobability_buffer
        )
        min_advantage = keras.ops.where(
            advantage_buffer > 0,
            (1 + clip_ratio) * advantage_buffer,
            (1 - clip_ratio) * advantage_buffer,
        )

        policy_loss = -keras.ops.mean(
            keras.ops.minimum(ratio * advantage_buffer, min_advantage)
        )
    policy_grads = tape.gradient(policy_loss, actor.trainable_variables)
    policy_optimizer.apply_gradients(zip(policy_grads, actor.trainable_variables))

    kl = keras.ops.mean(
        logprobability_buffer
        - logprobabilities(actor(observation_buffer), action_buffer)
    )
    kl = keras.ops.sum(kl)
    return kl

In [9]:
#critic: minimizes difference between predicted and actual value -- updates weights to make future predictions more accurate
@tf.function
def train_value_function(observation_buffer, return_buffer):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        value_loss = keras.ops.mean((return_buffer - critic(observation_buffer)) ** 2)
    value_grads = tape.gradient(value_loss, critic.trainable_variables)
    value_optimizer.apply_gradients(zip(value_grads, critic.trainable_variables))

In [ ]:
# Hyperparameters of the PPO algorithm
steps_per_epoch = 4000
epochs = 30
gamma = 0.99
clip_ratio = 0.3
policy_learning_rate = 5e-4
value_function_learning_rate = 1e-3
train_policy_iterations = 150
train_value_iterations = 80
lam = 0.97
target_kl = 0.02
hidden_sizes = (64, 64)

# True to render the environment
render = False

In [11]:
# Initialize the environment and get the dimensionality of the
# observation space and the number of possible actions
env = gym.make("CartPole-v1")
observation_dimensions = env.observation_space.shape[0]
num_actions = env.action_space.n

# Initialize the buffer
buffer = Buffer(observation_dimensions, steps_per_epoch)

# Actor (Policy Network): Maximizes expected rewards by improving actions.
observation_input = keras.Input(shape=(observation_dimensions,), dtype="float32")
logits = mlp(observation_input, list(hidden_sizes) + [num_actions])
actor = keras.Model(inputs=observation_input, outputs=logits)

# Critic (Value Function): Helps stabilize learning by reducing variance.
value = keras.ops.squeeze(mlp(observation_input, list(hidden_sizes) + [1]), axis=1)
critic = keras.Model(inputs=observation_input, outputs=value)

# Optimizers: Adaptive Moment Estimation (Adam)
policy_optimizer = keras.optimizers.Adam(learning_rate=policy_learning_rate)
value_optimizer = keras.optimizers.Adam(learning_rate=value_function_learning_rate)

# Initialize the observation, episode return and episode length
observation, _ = env.reset()
episode_return, episode_length = 0, 0

In [ ]:
# Initialize rewards_per_epoch before training
rewards_per_epoch = []

# Iterate over the number of epochs
for epoch in range(epochs):
    # Initialize the sum of the returns, lengths and number of episodes for each epoch
    sum_return = 0
    sum_length = 0
    num_episodes = 0
    episode_return = 0 

    # Iterate over the steps of each epoch
    for t in range(steps_per_epoch):
        if render:
            env.render()

        # Get the logits, action, and take one step in the environment
        observation = observation.reshape(1, -1)
        logits, action = sample_action(observation)
        observation_new, reward, done, _, _ = env.step(action[0].numpy())
        episode_return += reward
        episode_length += 1

        # Get the value and log-probability of the action
        value_t = critic(observation)
        logprobability_t = logprobabilities(logits, action)

        # Store obs, act, rew, v_t, logp_pi_t
        buffer.store(observation, action, reward, value_t, logprobability_t)

        # Update the observation
        observation = observation_new

        # Finish trajectory if reached to a terminal state
        terminal = done
        if terminal or (t == steps_per_epoch - 1):
            last_value = 0 if done else critic(observation.reshape(1, -1))
            buffer.finish_trajectory(last_value)
            sum_return += episode_return
            sum_length += episode_length
            num_episodes += 1
            observation, _ = env.reset()
            episode_return, episode_length = 0, 0

    # Get values from the buffer
    (
        observation_buffer,
        action_buffer,
        advantage_buffer,
        return_buffer,
        logprobability_buffer,
    ) = buffer.get()

    # Update the policy and implement early stopping using KL divergence
    for _ in range(train_policy_iterations):
        kl = train_policy(
            observation_buffer, action_buffer, logprobability_buffer, advantage_buffer
        )
        if kl > 1.5 * target_kl:
            # Early Stopping
            break

    # Update the value function
    for _ in range(train_value_iterations):
        train_value_function(observation_buffer, return_buffer)

    # Print mean return and length for each epoch
    print(
        f" Epoch: {epoch + 1}. Mean Return: {sum_return / num_episodes}. Mean Length: {sum_length / num_episodes}"
    )
    rewards_per_epoch.append(sum_return / max(num_episodes, 1))  #Avoid ZeroDivisionError


save_path = "/tmp/rewards_per_epoch.npy"  # Use a writable directory

# check if directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save rewards only after all epochs are completed
np.save(save_path, np.array(rewards_per_epoch))

print(f"Rewards saved at: {save_path}")

 Epoch: 1. Mean Return: 21.978021978021978. Mean Length: 21.978021978021978
 Epoch: 2. Mean Return: 37.735849056603776. Mean Length: 37.735849056603776
 Epoch: 3. Mean Return: 68.96551724137932. Mean Length: 68.96551724137932
 Epoch: 4. Mean Return: 117.6470588235294. Mean Length: 117.6470588235294
 Epoch: 5. Mean Return: 200.0. Mean Length: 200.0
 Epoch: 6. Mean Return: 142.85714285714286. Mean Length: 142.85714285714286
 Epoch: 7. Mean Return: 444.44444444444446. Mean Length: 444.44444444444446
 Epoch: 8. Mean Return: 222.22222222222223. Mean Length: 222.22222222222223
 Epoch: 9. Mean Return: 1333.3333333333333. Mean Length: 1333.3333333333333
 Epoch: 10. Mean Return: 333.3333333333333. Mean Length: 333.3333333333333
 Epoch: 11. Mean Return: 166.66666666666666. Mean Length: 166.66666666666666
 Epoch: 12. Mean Return: 400.0. Mean Length: 400.0
 Epoch: 13. Mean Return: 181.8181818181818. Mean Length: 181.8181818181818
 Epoch: 14. Mean Return: 800.0. Mean Length: 800.0
 Epoch: 15. Mean 

In [1]:
import matplotlib
matplotlib.use("Agg")  # Use a non-GUI backend for GitHub Actions
import matplotlib.pyplot as plt
import numpy as np
import os

# Define the correct file path (Modify based on your saving location)
save_path = "/tmp/rewards_per_epoch.npy"  # 🔹 Use this if stored in /tmp/
# save_path = "/github/workspace/rewards_per_epoch.npy"  # 🔹 Use this if running on GitHub Actions

# Ensure the file exists before loading to prevent crashes
if os.path.exists(save_path):
    rewards_per_epoch = np.load(save_path)
else:
    raise FileNotFoundError(f"⚠️ File not found: {save_path}")

# Generate the training visualization
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(rewards_per_epoch) + 1), rewards_per_epoch, marker="o", linestyle="-", color="b")
plt.xlabel("Epochs")
plt.ylabel("Mean Return")
plt.title("PPO Training Performance on CartPole-v1")
plt.grid()
plt.ylim(bottom=0, top=max(rewards_per_epoch) * 1.2 + 1)

# Save the plot in a writable location
output_path = "/tmp/ppo_training.png"  # 🔹 Adjust based on the environment
# output_path = "/github/workspace/ppo_training.png"  # 🔹 Use this if in GitHub Actions

plt.savefig(output_path, dpi=300)

print(f" Plot saved at: {output_path}")

 Plot saved at: /tmp/ppo_training.png
